#### **DATA CLEANING: THE NUMBERS(TNDB)**
The Numbers' production budget data is crucial for calculating true return on investment (ROI), which is often more relevant than gross revenue alone. This enables YE Studios to identify which genres and budget ranges offer the best profit margins, not just the highest revenue. The budget-to-revenue ratio analysis helps determine the minimum viable investment for different film categories.

In [ ]:
#LOAD DATA:
import pandas as pd
import numpy as np
import gzip

tndb = pd.read_csv("../data/zippedData/tn.movie_budgets.csv.gz", compression = "gzip")
tndb.head()

In [ ]:
#CLEAN DATA:
##CLEAN COLUMN NAMES:Make column names lowercase and replace spaces with underscores, and remove parantheses.
tndb.columns = tndb.columns.str.strip().str.lower().str.replace(" ","_").str.replace("(","").str.replace(")","")
tndb.head()
tndb.dtypes

In [ ]:
#CLEAN DATES:
#Convert release date to datetime formart.
tndb["release_date"]= pd.to_datetime(tndb["release_date"], errors = "coerce")

#Extract year for analysis:
tndb["release_year"]= tndb["release_date"].dt.year
tndb.head()

In [ ]:
#CLEAN FINACIAL DATA:
#Remove $ sign and convert to numeric.
tndb["production_budget"]= tndb["production_budget"].str.replace("$","").str.replace(",","")
tndb["production_budget"] = pd.to_numeric(tndb["production_budget"], errors="coerce")


tndb["domestic_gross"]= tndb["domestic_gross"].str.replace("$","").str.replace(",","")
tndb["domestic_gross"] = pd.to_numeric(tndb["domestic_gross"], errors="coerce")


tndb["worldwide_gross"]= tndb["worldwide_gross"].str.replace("$","").str.replace(",","")
tndb["worldwide_gross"] = pd.to_numeric(tndb["worldwide_gross"], errors="coerce")

tndb.head()

In [ ]:
#CREATE NEW COLUMNS:
#Calcuate profit
tndb["profit"]= tndb["worldwide_gross"]- tndb["production_budget"]

#Calculate return on investement(ROI)
tndb["roi"] = (tndb["profit"] / tndb["production_budget"] * 100)

#Calulate profit margin
tndb["foreign_gross"]= tndb["worldwide_gross"]- tndb["domestic_gross"]

tndb.head()

In [ ]:
#HANDLE DUPLICATES AND REMOVE MISSING VALUES:
#Sort by worldwide_gross and keep highest grosssing movie for each title.
tndb = tndb.sort_values("worldwide_gross", ascending= False)
tndb = tndb.drop_duplicates(subset= "movie", keep = "first")

#Drop rows where budget or worldwide_gross is missing
tndb = tndb.dropna(subset=["production_budget", "worldwide_gross"])
tndb.head()

In [ ]:
##FINAL CHECK OF CLEANED DATA:
#View cleaned data
tndb.head()

In [ ]:
tndb.isnull().sum()

In [ ]:
##Saving cleaned Data:
tndb_movies_cleaned = tndb.copy()
tndb_movies_cleaned.to_csv('../data/cleanedData/tndb_cleaned_data.csv', index=False)

## Data Cleaning: TMDB Movies

The TMDB dataset provides vital metrics on movie popularity and genre classification. To ensure the data is ready for analysis, the following preprocessing steps were performed:

### 1. Removing Redundancy
The `Unnamed: 0` column was dropped, as it represents a duplicate index and does not add analytical value. Removing this column helps streamline the dataframe.

### 2. Temporal Feature Engineering
The `release_date` column was converted to a standard `datetime` format. From this, a new feature, `release_year`, was extracted to enable time-series and trend-based analysis of movie releases.

### 3. Genre Data Parsing
The `genre_ids` column is stored as a string representation of a list (e.g., `"[12, 14]"`). The `ast.literal_eval` function was used to convert these strings into actual Python lists. This transformation allows the genre IDs to be mapped to their corresponding genre names (such as *Action* or *Comedy*) in later stages of the analysis.


In [ ]:
# ==========================================
# DATA CLEANING: TMDB Movies
# ==========================================

import ast
import pandas as pd
# Load the dataset directly using the relative path
tmdb_movies = pd.read_csv('../data/zippedData/tmdb.movies.csv.gz')
# Preview the first few rows to ensure successful loading
tmdb_movies.head()

# 1. Drop redundant index column
if 'Unnamed: 0' in tmdb_movies.columns:
    tmdb_movies.drop(columns=['Unnamed: 0'], inplace=True)

# 2. Feature Engineering: Convert release_date to Datetime and extract Year
tmdb_movies['release_date'] = pd.to_datetime(tmdb_movies['release_date'])
tmdb_movies['release_year'] = tmdb_movies['release_date'].dt.year

# 3. Clean Genre IDs 
# They are strings like "[12, 14]". We turn them into actual Python lists.
tmdb_movies['genre_ids'] = tmdb_movies['genre_ids'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

## Data Export

After completing feature engineering and data cleaning, the processed TMDB dataset was saved as a CSV file. This ensures the cleaned data is readily available for the exploratory data analysis (EDA) phase, eliminating the need to rerun the preprocessing and cleaning steps.


In [ ]:
tmdb_movies_cleaned = tmdb_movies.copy()
tmdb_movies_cleaned.to_csv('../data/cleanedData/tmdb_cleaned_data.csv', index=False)

# IMDB DATASET

#### Introduction 
The cleaned IMDB dataset provides standardized information on movie titles, genres, runtime, release year, ratings, and vote counts, prepared for analysis. The cleaning process involved merging relevant tables, handling missing values, correcting data types, and removing duplicates to improve data quality and consistency.

This refined dataset supports reliable analysis of movie characteristics and audience reception and will be used alongside other sources to identify trends and inform recommendations on the types of films the studio should produce.

#### Import libraries

In [ ]:
import pandas as pd
import sqlite3

#### PATH 

In [ ]:
data_path = '../data/zippedData/' # Set data path

#### Connect to IMDB database

In [ ]:
conn = sqlite3.connect(data_path + "im.db")

What this does

Opens the IMDB SQLite database

Creates connection called conn

# LOADING IMBD DATA

The IMDB dataset consists of two separate tables: movie_basics and movie_ratings. These tables were merged using the movie_id column to create a unified dataset containing movie titles, genres, runtime, and ratings. Merging the datasets allows for comprehensive analysis of movie characteristics and performance.

#### Load first IMDB table (movie_basics)

In [ ]:
movie_basics = pd.read_sql("SELECT * FROM movie_basics", conn)
movie_basics.head()

#### Load second IMDB table (movie_ratings)

In [ ]:
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings", conn)
movie_ratings.head()

#### Merge both tables

In [ ]:
imdb_movies = movie_basics.merge(movie_ratings, on="movie_id")
imdb_movies.head()

# CLEANING

In [ ]:
imdb_movies.info()

The imdb_movies.info() function was used to inspect the dataset structure and identify missing values. Columns such as runtime_minutes and genres had fewer non-null entries compared to the total number of rows, indicating missing data that required cleaning before analysis.

#### 1. Remove columns we don’t need
We don’t need original_title.

In [ ]:
imdb_movies = imdb_movies.drop(columns=['original_title'])

#### 2. Handle missing runtime
runtime_minutes → 66236 non-null But total rows = 73856 So about 7,000 missing.

We cannot analyze runtime if missing.

Remove those rows:

In [ ]:
imdb_movies = imdb_movies.dropna(subset=['runtime_minutes'])

The runtime_minutes column contained several missing values. Since runtime is a key variable in analyzing movie performance and the dataset remained sufficiently large after removal, rows with missing runtime values were dropped to ensure accuracy and consistency in analysis.

#### 3. Remove missing genres

In [ ]:
imdb_movies = imdb_movies.dropna(subset=['genres'])

The genres column contained missing values. Since genre is a key variable for identifying movie categories and determining performance by film type, rows with missing genre information were removed to ensure accurate analysis

#### 4. Convert runtime to integer

In [ ]:
imdb_movies['runtime_minutes'] = imdb_movies['runtime_minutes'].astype(int)

The runtime_minutes column was converted from float to integer to reflect actual movie runtime values in whole minutes and ensure consistency for analysis.

#### 5. Remove duplicates

Sometimes same movie appears twice.

In [ ]:
imdb_movies = imdb_movies.drop_duplicates(subset='primary_title')

6. Final cleaned dataset check

In [ ]:
imdb_movies.info()

- no missing runtime

- no missing genres

- clean dataset

#### 7. convert genres to lowercase

In [ ]:
imdb_movies['genres'] = imdb_movies['genres'].str.lower()

The IMDB dataset has been cleaned by removing unnecessary columns, handling missing values and converting data types. This ensured the dataset was accurate and suitable for analysis...

In [ ]:
imdb_movies_cleaned = imdb_movies.copy()
imdb_movies_cleaned.to_csv('../data/cleanedData/imdb_cleaned_data.csv', index = False)
